# Problème : Organisation d’un championnat

**Auteurs**
- 28627904 LUONG Ethan
- 21304385 PHAM Louis-Antoine

**Objectif.** Le but de ce TME est de programmer la grille de match d’un championnat. On considère qu’il y a $n_e$ équipes participantes. Les matchs de championnat peuvent se d´erouler le mercredi ou le dimanche. Le championnat dure (au maximum) ns semaines (avec 2 matchs par semaine), soit $n_j = 2\times n_s$ jours de match.

Les matchs se font sur le terrain de l’une des deux équipes, qui est considérée comme jouant *à domicile*. L’autre équipe joue *à l’extérieur*. Etant donné les déplacements et la fatigue du match, chaque équipe ne peut jouer plus d’un match par jour. Sur la durée du championnat, chaque équipe doit rencontrer l’ensemble des autres équipes une fois *à domicile* et une fois *à l’extérieur*, soit exactement 2 matchs par équipe adverse.

Le but est de produire un planning des matchs, indiquant pour chaque jour quelles équipes s’affrontent en précisant où ont lieu les matchs.

Afin d’avoir une solution générique, paramétrable par $n_j$ et $n_e$, et en raison du nombre important de variables propositionnelles et de contraintes à considérer, les exercices suivants ont pour objectif d’écrire des fonctions permettant de générer le fichier DIMACS représentant le problème. Le langage de programmation est au choix.

## Modélisation

Chaque équipe correspond à un numéro entre $0$ et $n_e−1$. De même, chaque jour de match correspond à un numéro entre $0$ et $n_j−1$. On considère que le championnat commence un mercredi. Les jours pairs ($0$ compris), correspondent donc à des mercredis, tandis que les jours impairs correspondent à des dimanches.

On représente par des variables propositionnelles $m{j,x,y}$ le fait qu’il y ait (ou non) un match entre l’´equipe $x$, jouant *à domicile*, et l’équipe $y$ au jour $j$.

Au format DIMACS toutes les variables doivent être numérotées. On propose de coder la variable $m{j,x,y}$ par $v_k$ où $k = j\times n_e^2 + x\times n_e + y + 1$.

In [ ]:
# Il y a mjxy = nj*ne*ne sans la contrainte x != y

def codage(ne, nj, j, x, y):
    assert(ne > 0)
    assert(j <= nj-1 and j >= 0)
    assert(x <= ne-1 and x >= 0)
    assert(x <= ne-1 and x >= 0)
    return j*ne**2 + x*ne + y + 1

k = codage(3, 2, 1, 2, 1)
print('Codage : ', k)

# On peut voir k comme un mot en base ne avec 3 chiffres : j, x, y
def decodage(k, ne):
    assert(ne > 0)
    y = (k-1) % ne
    x = ((k-1) // ne) % ne
    j = (k-1) // ne**2
    return j, x, y

print('Decodage : ', decodage(k, 3))

In [ ]:
print('Tests des intervalles de valeurs :')
try:
    codage(0, 2, 2, 2, 1)
except AssertionError:
    print('\tIl faut ne > 0')

try:
    codage(1, 2, 2, 2, 1)
except AssertionError:
    print('\tx et y doivent être strictement plus petits que ne')

try:
    codage(3, 2, 4, 2, 1)
except AssertionError:
    print('\tj doit être strictement plus petit que nj')

In [ ]:
print('Tests des valeurs de codage')
assert(codage(3, 2, 0, 0, 0) == 1)
assert(codage(3, 2, 1, 2, 1) == 17)
assert(codage(3, 2, 1, 0, 0) == 10)
print('Tests réussis !')

In [ ]:
print('Tests décodage')
assert(decodage(codage(3, 3, 2, 0, 0), 3) == (2, 0, 0))
assert(decodage(codage(3, 6, 5, 2, 1), 3) == (5, 2, 1))
print('Tests réussis !')

## Génération d'un planning de matchs

Le but de cet exercice est de résoudre le problème par une méthode SAT directe. Il s’agit donc de traduire et encoder le problème en SAT, d’utiliser glucose pour résoudre le problème ainsi traduit, puis de décoder le modèle fourni en un format adapté au problème et lisible.

In [ ]:
def au_moins(L):
    assert(not any([v <= 0 for v in L]))
    if len(L) == 0:
        return ''
    return ' '.join([str(v) for v in L]) + ' 0\n'

# Encodage par paires
def au_plus(L):
    assert(not any([v <= 0 for v in L]))
    contrainte = ''
    for i in range(len(L)):
        for j in range(i+1, len(L)):
            contrainte += f'{-L[i]} {-L[j]} 0\n'
    return contrainte

In [ ]:
print('Tests de au_moins')
assert(au_moins([]) == '')
assert(au_moins([1, 2, 3]) == '1 2 3 0\n')
print('Tests réussis !')

In [ ]:
print('Tests de au_plus')
assert(au_plus([]) == '')
assert(au_plus([1, 2, 3]) == '-1 -2 0\n-1 -3 0\n-2 -3 0\n')
print('Tests réussis !')

In [ ]:
def encoderC1(ne, nj):
    assert(ne > 0 and nj > 0)
    C1 = ''
    for j in range(nj):
        for x in range(ne):
            mjxy = [codage(ne, nj, j, x, y) for y in range(ne) if y != x] + [codage(ne, nj, j, y, x) for y in range(ne) if y != x]
            C1 += au_plus(mjxy)
    return C1, len(C1.split('\n'))-1

C1, n1 = encoderC1(3, 4)
print('Nombres de contraintes C1 : ', n1)
print('Contraintes C1 : ', C1)

In [ ]:
def encoderC2(ne, nj):
    assert(ne > 0 and nj > 0)
    C2 = ''
    for x in range(ne):
        for y in range(x+1, ne):
            mjxy = [codage(ne, nj, j, x, y) for j in range(nj)]
            mjyx = [codage(ne, nj, j, y, x) for j in range(nj)]
            C2 += au_moins(mjxy) + au_plus(mjxy)
            C2 += au_moins(mjyx) + au_plus(mjyx)
    return C2, len(C2.split('\n'))-1

C2, n2 = encoderC2(3, 4)
print('Nombres de contraintes C2 : ', n2)
print('Contraintes C2 : ', C2)

In [ ]:
import os

def encoderC0(ne, nj):
    assert(ne > 0 and nj > 0)
    L = [codage(ne, nj, j, x, x) for x in range(ne) for j in range(nj)]
    C0 = ' 0\n'.join([str(-v) for v in L]) + ' 0\n'
    return C0, len(C0.split('\n'))-1

def encoder(ne, nj):
    assert(ne > 0 and nj > 0)
    C0, n0 = encoderC0(ne, nj)
    C1, n1 = encoderC1(ne, nj)
    C2, n2 = encoderC2(ne, nj)
    return f'p cnf {nj*ne**2} {n0+n1+n2}\n{C0}{C1}{C2}'

def dico_equipes(filename='equipes.txt'):
    dico = {}
    with open(filename, 'r') as f:
        lignes = f.readlines()
        for i, ligne in enumerate(lignes):
            dico[i] = ligne
        return dico

def decoder(ne, equipes):
    equipes = dico_equipes(equipes)
    solution = ''
    with open('resultats.txt', 'r') as f:
        lignes = f.readlines()
        for ligne in lignes:
            tokens = ligne.split()
            if len(tokens) == 0:
                continue
            if tokens[0] == 's' and tokens[1] == 'UNSATISFIABLE':
                return 'UNSAT'
            elif tokens[0] == 'v':
                for v in tokens[1:]:
                    if int(v) > 0:
                        j, x, y = decodage(int(v), ne)
                        solution += f'Jour {j} : {equipes[x][:-1]} (dom.) vs {equipes[y][:-1]} (ext.)\n'
    return solution
            
def programme(ne, nj, equipes):
    os.system(f'echo "{encoder(ne, nj)}" | ../glucose/simp/glucose -model > resultats.txt')
    return decoder(ne, equipes)

In [ ]:
print(programme(3, 4, 'equipes.txt'))

## Optimisation du nombre de jours

In [ ]:
import signal

class TimeoutError(Exception):
    pass

def signal_handler(signum, frame):
    raise TimeoutError

def optimiser_nj_selon_ne(equipes, ne_min=3, ne_max=11, timeout=10):
    for ne in range(ne_min, ne_max+1):
        nj_min = 2*(ne-1)
        nj_max = ne*(ne-1)
        nj_inf = nj_min
        while nj_min <= nj_max:
            nj = (nj_max + nj_min) // 2
            signal.signal(signal.SIGALRM, signal_handler)
            signal.alarm(timeout)
            try:
                solution = programme(ne, nj, equipes)
            except TimeoutError:
                solution = 'Timeout'
            finally:
                signal.alarm(0)
                if solution != 'UNSAT':
                    has_timeout = solution == 'Timeout'
                    nj_max = nj - 1
                    nj_inf = nj
                else:
                    nj_min = nj + 1
        print(f'Pour ne = {ne}, on a nj_inf = {nj_inf} ({has_timeout and "Timeout" or "OK"})')

optimiser_nj_selon_ne('equipes.txt')

## Équilibrer les déplacements et les week-ends

In [ ]:
from itertools import combinations

def au_plus_etendu(L, k):
    contrainte = ''
    combinaisons = list(combinations(L, k+1))
    for sous_liste in combinaisons:
        contrainte += ' '.join([str(-v) for v in sous_liste]) + ' 0\n'
    return contrainte

def au_moins_etendu(L, k):
    N = len(L)
    return au_plus_etendu([-v for v in L], N-k)

print(au_plus_etendu([1, 2, 3, 4], 2))
print(au_moins_etendu([1, 2, 3, 4], 2))

In [ ]:
import math

def encoderC3(ne, nj, p_ext=0.5, p_dom=0.4):
    assert(ne > 0 and nj > 0)
    assert(p_ext >= 0 and p_ext <= 1)
    assert(p_dom >= 0 and p_dom <= 1)
    C3 = ''
    for x in range(ne):
        L_ext = [codage(ne, nj, dimanche, y, x) for y in range(ne) if y != x for dimanche in range(1, nj, 2)]
        C3 += au_moins_etendu(L_ext, math.ceil(p_ext*(ne-1)))
        L_dom = [codage(ne, nj, dimanche, x, y) for y in range(ne) if y != x for dimanche in range(1, nj, 2)]
        C3 += au_moins_etendu(L_dom, math.ceil(p_dom*(ne-1)))
    return C3, len(C3.split('\n'))-1

C3, n3 = encoderC3(3, 4)
print('Nombres de contraintes C3 : ', n3)
print('Contraintes C3 : ', C3)

In [ ]:
def encoderC4(ne, nj):
    assert(ne > 0 and nj > 0)
    C4 = ''
    for x in range(ne):
        for j in range(nj-2):
            L_ext = [codage(ne, nj, j, y, x) for y in range(ne) if y != x] + [codage(ne, nj, j+1, y, x) for y in range(ne) if y != x] + [codage(ne, nj, j+2, y, x) for y in range(ne) if y != x]
            C4 += au_plus_etendu(L_ext, 2)
            L_dom = [codage(ne, nj, j, x, y) for y in range(ne) if y != x] + [codage(ne, nj, j+1, x, y) for y in range(ne) if y != x] + [codage(ne, nj, j+2, x, y) for y in range(ne) if y != x]
            C4 += au_plus_etendu(L_dom, 2)
    return C4, len(C4.split('\n'))-1

C4, n4 = encoderC4(3, 4)
print('Nombres de contraintes C4 : ', n4)
print('Contraintes C4 : ', C4)

In [ ]:
def encoder_etendu(ne, nj):
    assert(ne > 0 and nj > 0)
    C0, n0 = encoderC0(ne, nj)
    C1, n1 = encoderC1(ne, nj)
    C2, n2 = encoderC2(ne, nj)
    C3, n3 = encoderC3(ne, nj)
    C4, n4 = encoderC4(ne, nj)
    return f'p cnf {nj*ne**2} {n0+n1+n2+n3+n4}\n{C0}{C1}{C2}{C3}{C4}'

def programme_etendu(ne, nj, equipes):
    os.system(f'echo "{encoder_etendu(ne, nj)}" | ../glucose/simp/glucose -model > resultats.txt')
    return decoder(ne, equipes)

In [ ]:
print(programme_etendu(3, 9, 'equipes.txt'))